In [ ]:
%matplotlib inline
# Importing necessary packages:
from glob import glob
import pandas as pd
import datetime as dt
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams, cycler
import matplotlib.lines as lines
from collections import OrderedDict
from matplotlib.legend import Legend
import matplotlib.patches as mpatches
import matplotlib.lines as mlines


import seaborn as sns
from PipeLine import *

In [ ]:
csv_files = glob("../sumrule_data/N*-analyze_bug/N*.csv")
geometry = 'cylinder'
df_name = 'all_in_one-ens_avg'
ext=['-properties-ens_avg.csv']
ens_evg_properties_files = PipeLine.file_reader(csv_files,extensions=ext)
ens_avg_properties_df = PipeLine.df_of_properties(ens_evg_properties_files, df_name, geometry, to_file=True,index_col=0)
ens_avg_properties_df = ens_avg_properties_df.round(4)
ens_avg_properties_df['fsd_normalized'] = 0.0
ens_avg_properties_df['gyr_normalized'] = 0.0
ens_avg_properties_df['rflory_normalized'] = 0.0
ens_avg_properties_df['vfrc_c_normalized'] = (ens_avg_properties_df['dmon'] * ens_avg_properties_df['vfrc_crowd']) / ens_avg_properties_df['dcrowd'] 
unique_simulations = [list(input_set) for input_set in list(ens_avg_properties_df.groupby(['nmon','dcyl','dcrowd']).size().index)]
for nmon, dcyl, dcrowd in unique_simulations:
    condition = (ens_avg_properties_df['nmon'] == nmon) & (ens_avg_properties_df['dcyl'] == dcyl) & (ens_avg_properties_df['dcrowd'] == dcrowd) & (ens_avg_properties_df['vfrc_crowd'] == 0)
    fsd = ens_avg_properties_df[condition]['fsd'].values[0]
    gyr = ens_avg_properties_df[condition]['gyr'].values[0]
    rflory = ens_avg_properties_df[condition]['rflory'].values[0]
    cond_normal = (ens_avg_properties_df['nmon'] == nmon) & (ens_avg_properties_df['dcyl'] == dcyl) & (ens_avg_properties_df['dcrowd'] == dcrowd)
    ens_avg_properties_df.loc[cond_normal,'fsd_normalized'] = ens_avg_properties_df.loc[cond_normal,'fsd'] / fsd
    ens_avg_properties_df.loc[cond_normal,'gyr_normalized'] = ens_avg_properties_df.loc[cond_normal,'gyr'] / gyr
    ens_avg_properties_df.loc[cond_normal,'rflory_normalized'] = ens_avg_properties_df.loc[cond_normal,'rflory'] / rflory

ens_avg_properties_df['vfrc_crowd'] = ens_avg_properties_df['vfrc_crowd'].round(decimals=3)
ens_avg_properties_df['vfrc_crowd_eff'] = (np.pi * ens_avg_properties_df['dcrowd'] ** 3 / 6) * ens_avg_properties_df['ncrowd'] / ((np.pi / 4 * (ens_avg_properties_df['dcyl']-ens_avg_properties_df['dcrowd']) ** 2) * ens_avg_properties_df['lcyl'])
ens_avg_properties_df['vfrc_c_eff_normalized'] = (ens_avg_properties_df['dmon'] * ens_avg_properties_df['vfrc_crowd_eff']) / ens_avg_properties_df['dcrowd'] 

dcrowds = list(ens_avg_properties_df.dcrowd.sort_values().drop_duplicates())
vfrc_crowds = list(ens_avg_properties_df.vfrc_crowd.sort_values().drop_duplicates())
vfrc_colors = sns.color_palette("coolwarm", len(vfrc_crowds))
dcyls = list(ens_avg_properties_df.dcyl.sort_values().drop_duplicates())
nmons = list(ens_avg_properties_df.nmon.sort_values().drop_duplicates())

line_styles = ['--',':','-.',(0, (3, 1, 1, 1, 1, 1))]
ls_cycle = cycler('ls', line_styles) # equal to len(dcrowds)

line_widths = np.arange(1,3,1)
lw_cycle = cycler('lw', line_widths) # equal to len(nmons

vfrc_crowds_selected = [0.0,0.1,0.2,0.3,0.4]
vfrc_colors_selected = sns.color_palette("Set2", len(vfrc_crowds_selected))#,as_cmap=True)
#vfrc_colors_selected = sns.color_palette("coolwarm", len(vfrc_crowds_selected))#,as_cmap=True)
vfrc_colors_selected

In [ ]:
xcol="vfrc_crowd"
df=ens_avg_properties_df
fontsize=20
colors = sns.color_palette(n_colors=4, palette="Set2", desat=1)
leg_labels = [r'$\frac{D}{a}$ (Line color)', '15.0', '20.0', '25.0','30.0', r'$N$ (Line size)', 1000, 2000, r'$\frac{a_c}{a}$ (Marker type)', '1.0', '2.0', '4.0']
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_crowd_eff", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_crowd", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_c_eff_normalized", leg_labels, colors, fontsize=20)
PipeLine.chainsize_plot(ens_avg_properties_df, "vfrc_c_normalized", leg_labels, colors, fontsize=20)

In [ ]:
csv_files = glob("../sumrule_data/N*-analyze_all/N*.csv")

ext=['-rHists-ens_avg.csv']
rHist_ens_avg_csvs = PipeLine.file_reader(csv_files,extensions=ext)
xname = 'r'
yname = 'hist_r'
divider = 'dcyl'
rHist_ens_avg_dfs = PipeLine.organize_hists(rHist_ens_avg_csvs,xname,yname,divider)

ext=['-zHists-ens_avg.csv']
zHist_ens_avg_csvs = PipeLine.file_reader(csv_files,extensions=ext)
xname = 'z'
yname = 'hist_z'
divider = 'lcyl'
zHist_ens_avg_dfs = PipeLine.organize_hists(zHist_ens_avg_csvs,xname,yname,divider)

ext=['-thetaHists-ens_avg.csv']
thetaHist_ens_avg_csvs = PipeLine.file_reader(csv_files,extensions=ext)
xname = 'theta'
yname = 'hist_theta'
divider = 'degree'
thetaHist_ens_avg_dfs = PipeLine.organize_hists(thetaHist_ens_avg_csvs,xname,yname,divider)

ext=['-rPhis-ens_avg.csv']
rPhis_ens_avg_csvs = PipeLine.file_reader(csv_files,extensions=ext)
xname = 'r'
yname = 'phi_r'
divider = 'dcyl'
rPhis_ens_avg_dfs = PipeLine.organize_hists(rPhis_ens_avg_csvs,xname,yname,divider)

ext=['-rRhos-ens_avg.csv']
rRhos_ens_avg_csvs = PipeLine.file_reader(csv_files,extensions=ext)
xname = 'r'
yname = 'rho_r'
divider = 'dcyl'
rRhos_ens_avg_dfs = PipeLine.organize_hists(rRhos_ens_avg_csvs,xname,yname,divider)

In [ ]:
# chain size in one ensemble
csv_files = glob("../N2000D25.0ac1*-analyze_bug/N*.csv")
#sizename = 'rFlory
#sizename = 'gyr'
sizename = 'fsd'
ext=['-'+sizename+'_t.csv']
fsd_csvs = PipeLine.file_reader(csv_files,extensions=ext)
xname = 't'
fsd_dfs = {}
for idx, csvfile in enumerate(fsd_csvs):
    dummdy_df = pd.read_csv(csvfile[0],index_col=0)
    firstcol_name = list(dummdy_df.columns)[0]
    keyname = firstcol_name.split('ens')[0]
    col_names = []
    
    for col in dummdy_df.columns:
        cell_attrs = PipeLine.cellAttributes(col,warning=False)
        col_name = sizename + '_' + str(cell_attrs.ens)
        col_names.append(col_name)
    dummdy_df = pd.read_csv(csvfile[0],names=col_names,index_col=0,skiprows=1)
    fsd_dfs[keyname] = dummdy_df
    
phic_c = 0.300
for keyname, fsd_df in fsd_dfs.items():
    print(keyname)
    cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
    vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
    mean_sizes = []
    if vfrc_crowd_dummy == phic_c: 
        nmon = cell_attrs.nmon
        dcyl = cell_attrs.dcyl
        dcrowd = cell_attrs.dcrowd
        fsd_df_plot = fsd_df
        mean_size = fsd_df.loc[40000:].mean()
        mean_sizes.append(mean_size)
        print(mean_size)
        
vfrc_crowds_selected = [0.1,0.2,0.3,0.4]
fig, axis = plt.subplots(nrows=1,ncols=1,figsize=(18.5, 10.5),sharex=True)
fontsize=22
nmon = 2000
dcyl = 25.0
dcrowd = 1.0

dict_of_ylabel = {'fsd':r'Futhermost distance, $L_{fsd}(t)$',
                 'gyr':r'Radius of gyration, $R_{G}(t)$',
                 'rFlory':r'End-to-end distance, $R_F(t)$'}

dict_of_ylimits = {'fsd':(75,250,25,5),
                 'gyr':(10,80,10,1),
                 'rFlory':(50,250,50,5)}

ens_labels = ['ensemble '+col.split('_')[-1] for col in fsd_df_plot.columns]
sns.lineplot(data=fsd_df_plot,ax=axis)
PipeLine.xticks(axis,(0,70000,5000,500),code=True,fontsize=fontsize-6)
xtick_labels  = np.arange(1000000,75000000,5000000)
axis.set_xticklabels(xtick_labels)
PipeLine.yticks(axis,dict_of_ylimits[sizename],code=True,fontsize=fontsize-6) 
axis.set_title(r'$N=$'+str(nmon)+', ${D}/{a}=$'+str(dcyl)+', ${a_c}/{a}=$'+str(dcrowd)+', $\phi_c=$'+str(vfrc_crowd_dummy),fontsize=fontsize)
axis.grid(True,ls=':',lw=1)
axis.tick_params(axis ='both',direction='in')
axis.set_ylabel(dict_of_ylabel[sizename],fontsize=fontsize)
axis.set_xlabel(r'Timestep, $t$',fontsize=fontsize)
lines, labels = axis.get_legend_handles_labels()
axis.legend(handles=lines, labels=ens_labels, title="Ensembles", loc='upper left', frameon=False,fontsize=fontsize-4,title_fontsize=fontsize)
fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = sizename+'-'+cell_attrs.filename+'-'+today+".png"
plt.savefig(fname,dpi=200)

In [ ]:
for keyname, fsd_df in fsd_dfs.items():
    print(keyname)
    cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
    vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
    mean_sizes = []
    if vfrc_crowd_dummy == phic_c and cell_attrs.ncrowd == 145688: 
        nmon = cell_attrs.nmon
        dcyl = cell_attrs.dcyl
        dcrowd = cell_attrs.dcrowd
        fsd_df_plot = fsd_df
        for col in fsd_df.columns:
            mean_size = np.array(fsd_df[col]).flatten()
            mean_size.sort()
            mean_size = mean_size[-5:]
            mean_sizes.append(mean_size)
            print(mean_size)

In [ ]:
def hist_plotter(hist_dfs,xname,yname,subfig_attrs,lw_attrs,ls_attrs,cl_attrs):
    fig, axes = plt.subplots(nrows=len(subfig_attrs)/2,ncols=2,figsize=(16,12),sharey='row')
    
    ls_cycle = cycler('ls', ls_attrs) # equal to len(dcrowds)
    lw_cycle = cycler('lw', lw_attrs) # equal to len(nmons
    cl_cycle = cycler(color = cl_attrs) 
    
    rcParams['axes.prop_cycle'] = cl_cycle
        
    axes = axes.flatten()
    fontsize=20
    for lw_attr, lw in zip(lw_attrs,lw_cycle):
        for ls_attr, ls in zip(ls_attrs,ls_cycle):
            for idx, (subfig_attr, axis) in enumerate(zip(subfig_attrs,axes)):
                
                axis.grid(True,ls=':',lw=1)
                axis.tick_params(axis ='both',direction='in')
                
                axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
                axis.set_title(r'${a_c}/{a}=$'+'${}$'.format(subfig_attr),fontsize=fontsize)
                #if idx > 1:
                 #   PipeLine.yticks(axis,(0.0,0.03,0.005,0.0005),code=True,fontsize=fontsize-6)
                #else:
                #   PipeLine.yticks(axis,(0.0,0.06,0.01,0.001),code=True,fontsize=fontsize-6)

                #if not idx % 2: 
                 #   axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize)
                #axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
                #PipeLine.xticks(axis,(0.0,1.0,0.1,0.01),code=True,fontsize=fontsize-6)
                lines = []
                for keyname, hist_df in hist_dfs.items():
                    cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                    vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                    if cell_attrs.nmon == nmon: 
                        linew = lw
                        
                    if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                        line = axis.plot('r_norm','phi_r_norm', data=rphi_ens_avg,alpha=0.8, **ls, **linew)
                        lines += line
    #axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
    #axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
    axis.set_xlabel(r'$\frac{2z}{L}$',fontsize=fontsize)
    color_patches = PipeLine.color_patcher(vfrc_colors_selected)
    color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                          title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
    axes[0].add_artist(color_legend)




    ls_handles = PipeLine.ls_handler(dcyls,line_styles)
    axes[1].legend(handles=ls_handles, title=r'${D}/{a}$ (line style)',ncol=1,title_fontsize=fontsize-4,
                   fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')

    lw_handles = PipeLine.lw_handler(nmons,line_widths)
    axes[2].legend(handles=lw_handles, title=r'$N$ (line width)',ncol=1,title_fontsize=fontsize-4,
                   fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')

    fig.tight_layout()
    plt.tight_layout()
    today = dt.date.today().strftime('%Y%m%d')
    fname = "phic_local_grid_dcrowd"+today+".pdf"
    plt.savefig(fname,dpi=200)

In [ ]:
# fsd_t-ens_avg
csv_files = glob("../N*-analyze_bug/N*.csv")
sizename = 'fsd'
#sizename = 'gyr'
#sizename = 'rFlory'
ext=['-'+sizename+'_t-ens_avg.csv']
fsd_csvs = PipeLine.file_reader(csv_files,extensions=ext)
xname = 't'
#divider = 'dcyl'
#rHist_ens_avg_dfs = PipeLine.organize_hists(rHist_ens_avg_csvs,xname,yname,divider)
fsd_dfs_ens_avg = {}
for idx, csvfile in enumerate(fsd_csvs):
    dummdy_df = pd.read_csv(csvfile[0],index_col=0)
    size_avg = dummdy_df.iloc[50000:,0].mean()
    keyname = list(dummdy_df.columns)[0].split('-')[0]
    cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
    #divider_dicts = {'dcyl':cell_attrs.dcyl, 'lcyl':cell_attrs.lcyl, 'degree': 360}
    dummdy_df = pd.read_csv(csvfile[0],names=[xname,yname],skiprows=1)
    #dummdy_df[xname+'_norm'] = 2 * dummdy_df[xname] / divider_dicts[divider]
    dummdy_df[sizename+'_norm'] = dummdy_df[yname] / size_avg
    fsd_dfs_ens_avg[keyname] = dummdy_df

# fsd_ens_avg at different dcrowd
vfrc_crowds_selected = [0.1,0.2,0.3,0.4]
vfrc_colors_selected = sns.color_palette("Set2", len(vfrc_crowds_selected))#,as_cmap=True)

xname = 't'
yname = sizename+'_norm'


ylabel_dict = {'fsd':r'$L_{fsd}(t)$','gyr':r'$R_{G}(t)$','rFlory':r'$R_{F}(t)$'}
ylabel_dict_long = {'fsd':r'Futhermost distance, $L_{fsd}(t)$',
                 'gyr':r'Radius of gyration, $R_{G}(t)$',
                 'rFlory':r'End-to-end distance, $R_F(t)$'}

# size ens evg different dcrowd
leg_pos = 'lower right'
fig, axes = plt.subplots(nrows=4,ncols=1,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=16
for nmon, lw in zip(nmons,lw_cycle):
    for dcyl, ls in zip(dcyls,ls_cycle):
        for idx, (dcrowd, axis) in enumerate(zip(dcrowds,axes)):
            axis.set_title(r'${a_c}/{a}=$'+'${}$'.format(dcrowd),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in') 
            axis.set_ylabel(ylabel_dict[sizename],fontsize=fontsize)
            PipeLine.xticks(axis,(0,70000,5000,500),code=True,fontsize=fontsize-6)
            xtick_labels  = np.arange(1000000,75000000,5000000)
            axis.set_xticklabels(xtick_labels)
            axis.set_xlabel(r'Timestep, $t$',fontsize=fontsize)
            lines = []
            for keyname, fsd_df in fsd_dfs_ens_avg.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: 
                    linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=fsd_df,alpha=0.8, **ls, **linew)
                    lines += line

axes[3].set_ylabel(ylabel_dict_long[sizename],fontsize=fontsize)

color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False)#,loc=leg_pos)
axes[0].add_artist(color_legend)

ls_handles = []
for dcyl, linestyle in zip(dcyls,line_styles):
    mline= mlines.Line2D([], [],  color='black', ls=linestyle, label=dcyl,lw=2)
    ls_handles.append(mline)

axes[1].legend(handles=ls_handles, title=r'${D}/{a}$ (line style)',ncol=1,title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False)#,loc=leg_pos)

lw_handles = []
for nmon, linewidth in zip(nmons,line_widths):
    mline= mlines.Line2D([], [],  color='black', lw=linewidth, label=nmon)
    lw_handles.append(mline)

axes[2].legend(handles=lw_handles, title=r'$N$ (line width)',ncol=1,title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False)#,loc=leg_pos)

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcrowd-"+yname+'-'+today+".png"
plt.savefig(fname,dpi=200)


# size_ens_avg at different dcyl
leg_pos = 'lower right'
fig, axes = plt.subplots(nrows=4,ncols=1,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=16
for nmon, lw in zip(nmons,lw_cycle):
    for dcrowd, ls in zip(dcrowds,ls_cycle):
        for idx, (dcyl, axis) in enumerate(zip(dcyls,axes)):
            axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            axis.set_ylabel(ylabel_dict[sizename],fontsize=fontsize)
            axis.set_xlabel(r'Timestep, $t$',fontsize=fontsize)
            PipeLine.xticks(axis,(0,70000,5000,500),code=True,fontsize=fontsize-6)
            xtick_labels  = np.arange(1000000,75000000,5000000)
            axis.set_xticklabels(xtick_labels)
            lines = []
            for keyname, fsd_df in fsd_dfs_ens_avg.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=fsd_df,alpha=0.8, **ls, **linew)
                    lines += line
axes[3].set_ylabel(ylabel_dict_long[sizename],fontsize=fontsize)

color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False)#,loc=leg_pos)
axes[0].add_artist(color_legend)
ls_handles = []
for dcrowd, linestyle in zip(dcrowds,line_styles):
    mline= mlines.Line2D([], [],  color='black', ls=linestyle, label=dcrowd,lw=2)
    ls_handles.append(mline)

axes[1].legend(handles=ls_handles, title=r'${a_c}/{a}$ (line style)',ncol=1,title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False)#,loc=leg_pos)

lw_handles = []
for nmon, linewidth in zip(nmons,line_widths):
    mline= mlines.Line2D([], [],  color='black', lw=linewidth, label=nmon)
    lw_handles.append(mline)

axes[2].legend(handles=lw_handles, title=r'$N$ (line width)',ncol=1,title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False)#,loc=leg_pos)
fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcyl-"+yname+'-'+today+".png"
plt.savefig(fname,dpi=200)

In [ ]:
xname = 'z_norm'
yname = 'hist_z_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))#,sharey='row')
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcyl, ls in zip(dcyls,ls_cycle):
        for idx, (dcrowd, axis) in enumerate(zip(dcrowds,axes)):
            axis.set_title(r'${a_c}/{a}=$'+'${}$'.format(dcrowd),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            axis.set_xlabel(r'$\frac{2z}{L}$',fontsize=fontsize)
            axis.set_ylabel(r'$\phi_m(z)$',fontsize=fontsize)
            #if idx > 1:
             #   PipeLine.yticks(axis,(0.0,0.03,0.005,0.0005),code=True,fontsize=fontsize-6)
            #else:
             #   PipeLine.yticks(axis,(0.0,0.06,0.01,0.001),code=True,fontsize=fontsize-6)

            #if not idx % 2: 
             #   axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize)
            PipeLine.xticks(axis,(-1*0.50,0.50,0.10,0.01),code=True,fontsize=fontsize-6)
            lines = []
            for keyname, rphi_ens_avg in zHist_ens_avg_dfs.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=rphi_ens_avg,alpha=0.8, **ls, **linew)
                    lines += line

PipeLine.yticks(axes[0],(0.0,0.004,0.0005,0.00005),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[1],(0.0,0.004,0.0005,0.00005),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[2],(0.0,0.008,0.001,0.0001),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[3],(0.0,0.01,0.002,0.0002),code=True,fontsize=fontsize-6,decimals=4)

color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

ls_handles = PipeLine.ls_handler(dcyls,line_styles)
axes[1].legend(handles=ls_handles, title=r'${D}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcrowd-"+yname+'-'+today+".png"
plt.savefig(fname,dpi=200)

In [ ]:
xname = 'z_norm'
yname = 'hist_z_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))#,sharey='row')
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcrowd, ls in zip(dcrowds,ls_cycle):
        for idx, (dcyl, axis) in enumerate(zip(dcyls,axes)):
            axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            axis.set_xlabel(r'$\frac{2z}{L}$',fontsize=fontsize)
            axis.set_ylabel(r'$\rho_m(z)$',fontsize=fontsize)
            #if idx > 1:
             #   PipeLine.yticks(axis,(0.0,0.03,0.005,0.0005),code=True,fontsize=fontsize-6)
            #else:
             #   PipeLine.yticks(axis,(0.0,0.06,0.01,0.001),code=True,fontsize=fontsize-6)

            #if not idx % 2: 
             #   axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize)
            PipeLine.xticks(axis,(-0.5,0.5,0.1,0.01),code=True,fontsize=fontsize-6)
            lines = []
            for keyname, rphi_ens_avg in zHist_ens_avg_dfs.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=rphi_ens_avg,alpha=0.8, **ls, **linew)
                    lines += line

PipeLine.yticks(axes[0],(0.0,0.004,0.0005,0.00005),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[1],(0.0,0.0035,0.0005,0.00005),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[2],(0.0,0.006,0.001,0.0001),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[3],(0.0,0.01,0.002,0.0002),code=True,fontsize=fontsize-6,decimals=4)

color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)

ls_handles = PipeLine.ls_handler(dcrowds,line_styles)
axes[1].legend(handles=ls_handles, title=r'${a_c}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcyl-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

In [ ]:
xname='r_norm'
yname='phi_r_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcyl, ls in zip(dcyls,ls_cycle):
        for idx, (dcrowd, axis) in enumerate(zip(dcrowds,axes)):
            axis.set_title(r'${a_c}/{a}=$'+'${}$'.format(dcrowd),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
            axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize)
            axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
            PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)
            lines = []
            for keyname, rphi_ens_avg in rPhis_ens_avg_dfs.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=rphi_ens_avg,alpha=0.8, **ls, **linew)
                    lines += line

PipeLine.yticks(axes[0],(0.0,0.05,0.005,0.0005),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[1],(0.0,0.06,0.006,0.0006),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[2],(0.0,0.03,0.003,0.0003),code=True,fontsize=fontsize-6,decimals=4)
PipeLine.yticks(axes[3],(0.0,0.03,0.003,0.0003),code=True,fontsize=fontsize-6,decimals=4)

#axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)
ls_handles = []
for dcrowd, linestyle in zip(dcrowds,line_styles):
    mline= mlines.Line2D([], [],  color='black', ls=linestyle, label=dcrowd,lw=2)
    ls_handles.append(mline)

ls_handles = PipeLine.ls_handler(dcyls,line_styles)
axes[1].legend(handles=ls_handles, title=r'${D}/{a}$ (line style)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')

lw_handles = PipeLine.lw_handler(nmons,line_widths)
axes[2].legend(handles=lw_handles, title=r'$N$ (line width)', ncol=1, title_fontsize=fontsize-4,
               fontsize=fontsize-6, framealpha=None, frameon=False, loc='upper right')


fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcrowd-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

In [ ]:
xname = 'r_norm'
yname = 'phi_r_norm'
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=22
for nmon, lw in zip(nmons,lw_cycle):
    for dcrowd, ls in zip(dcrowds,ls_cycle):
        for idx, (dcyl, axis) in enumerate(zip(dcyls,axes)):
            axis.set_title(r'${D}/{a}=$'+'${}$'.format(dcyl),fontsize=fontsize)
            axis.grid(True,ls=':',lw=1)
            axis.tick_params(axis ='both',direction='in')
            axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
            if idx > 1:
                PipeLine.yticks(axis,(0.0,0.03,0.003,0.0006),code=True,fontsize=fontsize-6)
            elif idx == 1: 
                PipeLine.yticks(axis,(0.0,0.04,0.004,0.0008),code=True,fontsize=fontsize-6)
            else:
                PipeLine.yticks(axis,(0.0,0.06,0.006,0.0012),code=True,fontsize=fontsize-6)

            axis.set_ylabel(r'$\phi_m(r)$',fontsize=fontsize)
            axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
            PipeLine.xticks(axis,(0.0,1.0,0.2,0.02),code=True,fontsize=fontsize-6)
            lines = []
            for keyname, rphi_ens_avg in rPhis_ens_avg_dfs.items():
                cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
                vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
                if cell_attrs.nmon == nmon: linew = lw
                if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                    line = axis.plot(xname,yname, data=rphi_ens_avg,alpha=0.8, **ls, **linew)
                    lines += line
#axes[0].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axes[2].set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
#axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
color_patches = PipeLine.color_patcher(vfrc_colors_selected)
color_legend = Legend(axes[0],handles=color_patches,labels= vfrc_crowds_selected,title=r'$\phi_c$ (color)',ncol=1,
                      title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')#,bbox_to_anchor=(1.02,-0.02))
axes[0].add_artist(color_legend)
ls_handles = []
for dcrowd, linestyle in zip(dcrowds,line_styles):
    mline= mlines.Line2D([], [],  color='black', ls=linestyle, label=dcrowd,lw=2)
    ls_handles.append(mline)

axes[1].legend(handles=ls_handles, title=r'${a_c}/{a}$ (line style)',ncol=1,title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')

lw_handles = []
for nmon, linewidth in zip(nmons,line_widths):
    mline= mlines.Line2D([], [],  color='black', lw=linewidth, label=nmon)
    lw_handles.append(mline)

axes[2].legend(handles=lw_handles, title=r'$N$ (line width)',ncol=1,title_fontsize=fontsize-4,fontsize=fontsize-6,framealpha=None,frameon=False,loc='upper right')



fig.tight_layout()
plt.tight_layout()
today = dt.date.today().strftime('%Y%m%d')
fname = "phic_local-grid_dcyl-"+yname+'-'+today+".pdf"
plt.savefig(fname,dpi=200)

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2,figsize=(16,12))
rcParams['axes.prop_cycle'] = cycler(color=vfrc_colors_selected) 
axes = axes.flatten()
fontsize=18
for dcrowd, ls in zip(dcrowds,ls_cycle):
    for dcyl, axis in zip(dcyls,axes):
        axis.set_ylabel(r'$\phi_c(r)$',fontsize=fontsize)
        axis.set_xlabel(r'$\frac{2r}{D}$',fontsize=fontsize)
        for keyname, rphi_ens_avg in rphis_ens_avg_dfs.items():
            cell_attrs = PipeLine.cellAttributes(keyname,warning=False)
            vfrc_crowd_dummy = np.round(cell_attrs.vfrc_crowd,decimals=3)
            if (cell_attrs.dcyl == dcyl) and (cell_attrs.dcrowd == dcrowd) and (vfrc_crowd_dummy in vfrc_crowds_selected):
                axis.plot('r_norm','phi_r_norm', data=rphi_ens_avg,**ls,lw=2)

In [ ]:
nmon = 2000
dcrowd = 4.0
ens_avg_properties_df

for direct in rhist_df.columns:
    path = os.path.join("./", direct)
    os.mkdir(path)

In [ ]:
fpath = hist_files[24]

histo_collections = np.loadtxt(fpath[0],dtype=np.int32)
bin_edges = np.around(np.loadtxt(fpath[1]),decimals=2)
cell_attrs = PipeLine.cellAttributes(fpath[0],geometry='cylinder',printname=True)
filename = cell_attrs.filename
cylindrical_shell_integrand = lambda r: 2 * np.pi * cell_attrs.dcyl * r

# the sum of rho is not equal to the bulk number density (r=infiity) natom/cell_vol
# this arises from the way we descritize the local number desnity.
rho = PipeLine.local_number_density(histo_collections, bin_edges, cylindrical_shell_integrand)

rmon = cell_attrs.dmon / 2.0
bin_centers = np.around((bin_edges[:-1] + bin_edges[1:]) / 2.0,decimals=2)

range_of_bins = PipeLine.bin_edge_index(rmon, bin_centers, bin_edges)
volume_shares = PipeLine.vol_shares(rmon, bin_centers, bin_edges, range_of_bins, PipeLine.sphere_cylinder_intersection)

# the sum of phi is not equal to the bulk volume fraction (r=infiity) natom*vol_per_atom/cell_vol
# this arises from the way we descritize the local volume fraction and the way we relate it to the local number density.
phi = PipeLine.local_volume_fraction(bin_centers, rho, volume_shares)

fig, ax = plt.subplots(nrows=1,ncols=1,figsize=(16,9))

fontsize = 24
color = 'tab:blue'
line1, = ax.plot(bin_centers, phi, label = r"$\phi_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
#ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)",fontsize=fontsize)
ax.set_ylabel(r"$\phi_m(r)$",fontsize=fontsize)
ax.tick_params(labelsize=24)

ax2 = ax.twinx()
color = 'tab:red'
line2, = ax2.plot(bin_centers, rho, label = r"$\rho_c(R=\infty)={:4.2f}$".format(cell_attrs.vfrc_crowd), markersize=0.1, c=color)
ax2.set_ylabel(r"$\rho_m(r)$",fontsize=fontsize)
ax2.tick_params(labelsize=24)

lines = [line1, line2]

ax.legend(lines, [l.get_label() for l in lines],loc=1,ncol=2,fontsize=fontsize)
#fig.legend()
picname = filename.split("_rHists")[0]+"_local"
plt.savefig(picname+'.pdf',dpi=300)

In [ ]:
fig, ax = plt.subplots(nrows=1,ncols=1)
nens = 0
hists = 0
phi_c = 0.0 

for filename in ens_avg_properties.filename:
    condition = ens_avg_properties['filename']==filename
    for col in rhist_dfs.columns:
        if col.split('ens')[0] == filename.split('_ens_avg')[0]:
            #print(rhist_dfs[col])
            PipeLine.histo_plotter(ax, rhist_dfs[col].values, lambda r: 2 * ens_avg_properties[condition].dcyl / 2.0 * r, rhist_dfs.index.values, label = r"$\phi_c={}$".format(ens_avg_properties[condition].vfrc_crowd), markersize=0.1)
    #ens_evg_rhist
    
#PipeLine.histo_plotter(ax, hists, lambda r: 2 * cell_attrs.dcyl / 2.0 * r, edges, label = r"$\phi_c={}$".format(phi_c), markersize=0.1)
ax.legend()
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")
ax.set_xlabel(r"Cylinder radius ($\frac{D}{2a}$)")